In [1]:
import os

In [2]:
from shutil import copyfile

In [3]:
from ase import io

In [4]:
from ase.calculators.espresso import Espresso

In [5]:
from pyiron.atomistics.job.atomistic import AtomisticGenericJob

In [6]:
from pyiron_base import GenericParameters, Settings, Executable

In [7]:
from pyiron import Project, pyiron_to_ase, ase_to_pyiron

In [8]:
s = Settings()

In [9]:
class QuantumEspressoInput(GenericParameters):
    def __init__(self, input_file_name=None):
        super(QuantumEspressoInput, self).__init__(
            input_file_name=input_file_name, table_name="input", comment_char="#"
        )

    def load_default(self):
        """
        Loading the default settings for the input file.
        """
        input_str = """\
kpoints [3,3,3]
tstress True
tprnfor True
"""
        self.load_string(input_str)

In [10]:
class QuantumEspresso(AtomisticGenericJob):
    def __init__(self, project, job_name):
        super(QuantumEspresso, self).__init__(project, job_name)
        self.__name__ = "QuantumEspresso"
        self._executable = Executable(
            codename=self.__name__, 
            path_binary_codes=s.resource_paths, 
            module="quantumespresso"
        )
        self.input = QuantumEspressoInput()
        self.pseudopotentials = {
            'Na': 'Na.pbe-spn-rrkjus_psl.1.0.0.UPF',
            'Cl': 'Cl.pbe-nl-rrkjus_psl.1.0.0.UPF'
        }
        
    def write_input(self):
        calc = Espresso(
            label="pyiron",
            pseudopotentials=self.pseudopotentials,
            tstress=self.input["tstress"], 
            tprnfor=self.input["tprnfor"], 
            kpts=self.input["kpoints"]
        )
        calc.directory = self.working_directory
        calc.write_input(atoms=pyiron_to_ase(self.structure))
      
    def collect_output(self):
        output = io.read(os.path.join(self.working_directory, 'pyiron.pwo'))
        with self.project_hdf5.open("output") as hdf5_output:
            for k in output.calc.results.keys():
                hdf5_output[k] = output.calc.results[k]
    
    def to_hdf(self, hdf=None, group_name=None):
        """
        Store the ExampleJob object in the HDF5 File

        Args:
            hdf (ProjectHDFio): HDF5 group object - optional
            group_name (str): HDF5 subgroup name - optional
        """
        super(QuantumEspresso, self).to_hdf(hdf=hdf, group_name=group_name)
        self._structure_to_hdf()
        with self.project_hdf5.open("input") as hdf5_input:
            self.input.to_hdf(hdf5_input)
            hdf5_input["potential"] = self.pseudopotentials

    def from_hdf(self, hdf=None, group_name=None):
        """
        Restore the ExampleJob object in the HDF5 File

        Args:
            hdf (ProjectHDFio): HDF5 group object - optional
            group_name (str): HDF5 subgroup name - optional
        """
        super(QuantumEspresso, self).from_hdf(hdf=hdf, group_name=group_name)
        self._structure_from_hdf()
        with self.project_hdf5.open("input") as hdf5_input:
            self.input.from_hdf(hdf5_input)
            self.pseudopotentials = hdf5_input["potential"]

In [11]:
pr = Project("test")

In [12]:
pr.remove_jobs_silently(recursive=True)

In [13]:
structure = pr.create_ase_bulk('NaCl', crystalstructure='rocksalt', a=6.0)

In [14]:
job = pr.create_job(QuantumEspresso, "qe")
job.structure = structure 
job.input

,Parameter,Value,Comment
0,kpoints,"[3,3,3]",
1,tstress,True,
2,tprnfor,True,


In [15]:
job.run()

The job qe was saved and received the ID: 1


In [16]:
job["output/energy"], job["output/forces"], job["output/stress"]

(-1742.9248893545657,
 array([[0., 0., 0.],
        [0., 0., 0.]]),
 array([0.01931804, 0.01931804, 0.01931804, 0.        , 0.        ,
        0.        ]))

In [17]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,ClNa,qe,/qe,/home/jovyan/,test/,2020-10-25 16:39:03.630310,2020-10-25 16:39:53.493553,49.0,pyiron@jupyter-jan-2djanssen-2dpyi-2duantum-2despresso-2dbaliuqjs#1,QuantumEspresso,0.1,None,None
